import 函式庫

In [53]:
import pandas as pd
import numpy as np
from datetime import datetime

讀取.csv

In [54]:
df = pd.read_csv("A_stat_data_112.csv", encoding="UTF-8")

資料清洗

In [55]:
df.dropna(subset=["completed_at_utc8"], inplace=True)
df.dropna(subset=["mission_difficulty"], inplace=True)
df = df[df['is_completed'] == 1]

求出任務發佈和任務完成之間的時間差

In [56]:
df['mission_datetime_utc8'] = pd.to_datetime(df['mission_datetime_utc8'], format="%a %b %d %H:%M:%S %Z %Y")
df['completed_at_utc8'] = pd.to_datetime(df['completed_at_utc8'], format="%a %b %d %H:%M:%S %Z %Y")

df['mission_continue_time'] = df['completed_at_utc8'] - df['mission_datetime_utc8']
df_sorted_asc = df.sort_values(by='mission_continue_time', ascending=True)

將任務名稱存成List

In [57]:
mission_list = df["mission_name"].unique()
mission_list = list(mission_list)

將用不到的欄刪掉，把dataset的容量降低

In [58]:
df.drop(["mission_semester_year", "mission_semester", "content_indicator_correct_rate"], axis=1, inplace=True)
df.drop(["comprehensive_indicator_correct_rate", "explanation_indicator_correct_rate"], axis=1, inplace=True)
df.drop(["extraction_indicator_correct_rate", "form_indicator_correct_rate"], axis=1, inplace=True)

印出保留下的欄位名稱

In [59]:
for i in df:
    print(i)

grade
mission_name
mission_difficulty
mission_field
mission_type
mission_datetime_utc8
mission_status
is_completed
completed_at_utc8
correct_rate
answer_duration_sec
mission_answer_count
first_correct_rate
user_id
mission_id
mission_continue_time


清洗資料

In [60]:
df.dropna(subset=["mission_difficulty", "answer_duration_sec"], inplace=True)

將任務難易度轉換成更直觀的數字

In [61]:
difficulty_mapping = {
    "易": 0.3,
    "中": 0.5,
    "難": 0.7
}

# 使用 map 方法進行轉換
df['mission_difficulty_num'] = df['mission_difficulty']
df['mission_difficulty_num'] = df['mission_difficulty_num'].map(difficulty_mapping)

將整理完的dataset輸出成.csv檔

In [62]:
df.to_csv("A_stat_data_112_cleared.csv", encoding="UTF-8")

做出排行榜所需的排序，並轉換整數

In [63]:
df.sort_values('mission_continue_time', inplace=True)
df.sort_values('answer_duration_sec', inplace=True)
df['answer_duration_sec'] = df['answer_duration_sec'].astype(int)

排行榜程式

In [ ]:
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title('PaGameO 排行榜') # 視窗標題
root.geometry('600x500') # 視窗大小
root.configure(bg='gray57') # 視窗背景顏色

def show(*e):
    a.set(value.get())  # Label 變數改變成選單內容，使用 get() 取值

def search():
    # 取得選單的當前值
    selected_value = value.get()
    # 清空結果區域的內容
    result_text.delete(1.0, tk.END)
    # 高效篩選與選擇值相關的資料
    filtered_df = df[df['mission_name'].str.contains(selected_value, na=False)]
    # 印出標題
    result_text.insert(tk.END, "用戶 ID\t\t\t完成時間\n")
    result_text.insert(tk.END, "-------------------------------------\n")
    # 印出篩選結果
    if filtered_df.empty:
        result_text.insert(tk.END, "無相關資料")
    else:
        for _, row in filtered_df.iterrows():
            result_text.insert(tk.END, f"{row['user_id']:<20}\t   {row['answer_duration_sec']}秒\n")
            result_text.insert(tk.END, "-------------------------------------\n")

# Label 變數
a = tk.StringVar()
a.set(mission_list[0])

label = tk.Label(root, textvariable=a)  # 建立 Label，文字為變數
label.pack()

# 建立 Combobox
value = tk.StringVar()
value.set(mission_list[0])  # 設定預設選項

combo = ttk.Combobox(root, textvariable=value, values=mission_list, state="readonly", width=50)
combo.pack()

# 綁定事件，當選項改變時執行 show 函數
value.trace('w', show)

# 搜尋按鈕
search_button = tk.Button(root, text="搜尋", command=search)
search_button.pack(pady=10)

# 顯示 DataFrame 的文字區域
result_text = tk.Text(root, height=15, width=60)
result_text.pack()

root.mainloop()


In [65]:
df2 = pd.read_csv("A_stat_data_112_cleared.csv", encoding="UTF-8")
df2

,Unnamed: 0,grade,mission_name,mission_difficulty,mission_field,mission_type,mission_datetime_utc8,mission_status,is_completed,completed_at_utc8,correct_rate,answer_duration_sec,mission_answer_count,first_correct_rate,user_id,mission_id,mission_continue_time,mission_difficulty_num
0,10,5,05/23 一起來看臺灣芒果【多元文化教育－異國美食】（初階）,易,生活知識,連續＋圖表,2024-05-23 08:00:00+00:00,完成任務,1,2024-07-03 11:51:42+00:00,0.0,10.0,4,0.0,3381812929067740697,8497080254375325300,41 days 03:51:42,0.3
1,11,5,05/27 無限產乳的牛【生命教育－食物奧秘】（初階）,中,議題,連續＋圖表,2024-05-27 08:00:00+00:00,完成任務,1,2024-07-04 12:00:13+00:00,0.4,31.0,5,0.4,3381812929067740697,-1529236269228583927,38 days 04:00:13,0.5
2,40,5,06/24 被誤會的塑膠袋？【能源教育－環境保育】（初階）,難,議題,連續,2024-06-24 08:00:00+00:00,完成任務,1,2024-07-10 09:27:09+00:00,0.2,15.0,4,0.2,3516683932418412104,325126197002900849,16 days 01:27:09,0.7
3,64,7,09/11 工讀生的勞保與健保【人權教育－勞動權益】（中階）,難,生活知識,連續＋圖表,2023-09-11 08:00:00+00:00,完成任務,1,2023-09-20 08:58:52+00:00,0.2,6.0,2,0.2,3034784768844041174,-7544460018076597178,9 days 00:58:52,0.7
4,65,7,09/11 工讀生的勞保與健保【人權教育－勞動權益】（中階）,難,生活知識,連續＋圖表,2023-09-11 08:00:00+00:00,完成任務,1,2023-09-14 17:50:33+00:00,0.4,11.0,4,0.4,8513761052470104222,-7544460018076597178,3 days 09:50:33,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426665,1909904,7,04/22 動物園該存在嗎？【環境教育－動物保育】（中階）,難,議題,非連續,2024-04-22 08:00:00+00:00,完成任務,1,2024-04-28 09:50:54+00:00,0.4,7.0,3,0.4,2131072479141623967,703618323232836566,6 days 01:50:54,0.7
426666,1909905,7,04/22 人與鼠的戰爭【環境教育－動物保育】（中階）,易,世界櫥窗,連續,2024-04-22 08:00:00+00:00,完成任務,1,2024-04-27 16:36:16+00:00,0.2,8.0,2,0.2,5544869316161769011,8809938774431070893,5 days 08:36:16,0.3
426667,1909909,7,04/22 動物園該存在嗎？【環境教育－動物保育】（中階）,難,議題,非連續,2024-04-22 08:00:00+00:00,完成任務,1,2024-04-22 21:47:13+00:00,0.0,15.0,2,0.0,-1161138445532532382,703618323232836566,0 days 13:47:13,0.7
426668,1909913,7,04/22 人與鼠的戰爭【環境教育－動物保育】（中階）,易,世界櫥窗,連續,2024-04-22 08:00:00+00:00,完成任務,1,2024-04-29 07:49:15+00:00,0.2,6.0,2,0.2,804447241632728037,8809938774431070893,6 days 23:49:15,0.3
